###Importing libraries

In [ ]:
import numpy as np
import pandas as pd

### Importing dataset




In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# dataset = pd.read_csv('./drive/MyDrive/Project1_Sentiment_Analysis/Half_data/Half_data.tsv', delimiter = '\t', quoting = 3)
# dataset = pd.read_csv('./drive/MyDrive/Project1_Sentiment_Analysis/Half_data/Half_data.csv', delimiter = ',', quoting = 0, encoding='cp1252')
# dataset = pd.read_csv('./drive/MyDrive/Project1_Sentiment_Analysis/New_try/MyProjWords.csv', delimiter = ',', quoting = 0, encoding='cp1252')
dataset = pd.read_csv('./drive/MyDrive/Project1_Sentiment_Analysis/With_actual_words/actual_word_list.csv', delimiter = ',', quoting = 0, encoding='cp1252')

In [ ]:
dataset.shape

(6833, 2)

In [ ]:
dataset.head()

,Review,Liked
0,absolutely,1
1,complete,1
2,completely,1
3,could,-1
4,definitely,1


### Data cleaning

In [ ]:
import re
import nltk

nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

all_stopwords = stopwords.words('english')
all_stopwords.remove('not')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
corpus=[]

for i in range(0, 6833):
  review = re.sub('[^a-zA-Z]', ' ', dataset['Review'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
  review = ' '.join(review)
  corpus.append(review)

In [ ]:
corpus

### Data transformation

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 1420)

In [ ]:
X = cv.fit_transform(corpus).toarray()
y = dataset.iloc[:, -1].values

In [ ]:
# Saving BoW dictionary to later use in prediction
import pickle
#bow_path = './drive/MyDrive/Project1_Sentiment_Analysis/already/c1_BoW_Sentiment_Model.pkl'
#bow_path = './drive/MyDrive/Project1_Sentiment_Analysis/Half_data/c1_BoW_Sentiment_Model.pkl'
#bow_path = './drive/MyDrive/Project1_Sentiment_Analysis/New_try/c1_BoW_Sentiment_Model.pkl'
bow_path = './drive/MyDrive/Project1_Sentiment_Analysis/With_actual_words/c1_BoW_Sentiment_Model.pkl'
pickle.dump(cv, open(bow_path, "wb"))

### Dividing dataset into training and test set

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

### Model fitting (Naive Bayes)

In [ ]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)

GaussianNB()

In [ ]:
# Exporting NB Classifier to later use in prediction
import joblib
#joblib.dump(classifier, './drive/MyDrive/Project1_Sentiment_Analysis/already/c2_Classifier_Sentiment_Model') 
#joblib.dump(classifier, './drive/MyDrive/Project1_Sentiment_Analysis/Half_data/c2_Classifier_Sentiment_Model') 
#joblib.dump(classifier, './drive/MyDrive/Project1_Sentiment_Analysis/New_try/c2_Classifier_Sentiment_Model') 
joblib.dump(classifier, './drive/MyDrive/Project1_Sentiment_Analysis/With_actual_words/c2_Classifier_Sentiment_Model') 

['./drive/MyDrive/Project1_Sentiment_Analysis/With_actual_words/c2_Classifier_Sentiment_Model']

###Model performance

In [ ]:
y_pred = classifier.predict(X_test)

from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
cm = confusion_matrix(y_test, y_pred)
print(cm)

accuracy_score(y_test, y_pred)


[[468 480  22   1]
 [  0   8   0   0]
 [  8 186 192   0]
 [  1   1   0   0]]


0.48866130212143377

In [ ]:
precision_score(y_test, y_pred, average='macro')

0.47254504725144825

In [ ]:
recall_score(y_test, y_pred, average = "macro")

0.4948466673425719

In [ ]:
f1_score(y_test, y_pred, average = "macro")

0.3274587253181042

48.86% - for With_actual_words accuracy

42.85% - for New_try - specific selected words from existing reviews

69.23% - Half data accuracy

72.77% - for Restaurant review data

